# DSCI 552 Homework 3

#### Name: Boyu Shen
#### GIthub Name: boyushen0701
#### USC ID: 3547352504

In [311]:
# import libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import os
from sklearn.utils import resample
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.metrics import roc_auc_score
import statsmodels.api as sm
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
from sklearn import metrics
from sklearn.feature_selection import f_classif
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

## 1. Time Series Classification

### (a) Download the AReM data

In [312]:
# get all names of activities
activities=["bending1","bending2","cycling","lying","sitting","standing","walking"]

In [313]:
# define data loading
def read_data(path):
    return pd.read_csv(path,skiprows=4,usecols=range(1,7))

In [314]:
# get train and test datapaths
col = ['time','avg_rss12','var_rss12','avg_rss13','var_rss13','avg_rss23','var_rss23']
totaldatapath = set(['dataset{0}.csv'.format(x) for x in range(1,16)])
testdatapath_1 = set(['dataset1.csv','dataset2.csv'])
testdatapath_2 = set(['dataset1.csv','dataset2.csv','dataset3.csv'])
traindatapath_1 = totaldatapath-testdatapath_1
traindatapath_2 = totaldatapath-testdatapath_2

### (b) Split training and test datasets

In [315]:
# get train and test dataset lists
traindata=[]
testdata=[]
train_dict = {}
test_dict = {}

# get train and test datasets for folder bending1 and bending2
for activity in activities[:2]:
    path="../data/AReM/{0}".format(activity)
    for file in os.listdir(path):
        # get rid of the dilimiter problem
        if activity == 'bending2' and file == 'dataset4.csv':
            df = pd.read_csv('../data/AReM/bending2/dataset4.csv',skiprows=4,delim_whitespace=True).reset_index()
            df.columns = col
            df = df.drop(columns=['time'])
            train_dict[activity+' '+file] = df
            traindata.append(df)
           
        else:
            if file in testdatapath_1:
                df = read_data("../data/AReM/{0}/{1}".format(activity,file))
                test_dict[activity+' '+file] = df
                testdata.append(df)
            if file in traindatapath_1:
                df = read_data("../data/AReM/{0}/{1}".format(activity,file))
                train_dict[activity+' '+file] = df    
                traindata.append(df)
                
# get train and test datasets for other folders
for activity in activities[2:]:
    path="../data/AReM/{0}".format(activity)
    for file in os.listdir(path):
        if file in testdatapath_2:
            df = read_data("../data/AReM/{0}/{1}".format(activity,file))
            test_dict[activity+' '+file] = df
            testdata.append(df)
        if file in traindatapath_2:
            df = read_data("../data/AReM/{0}/{1}".format(activity,file))
            train_dict[activity+' '+file] = df
            traindata.append(df)

### (c) Feature Extraction

#### i. Research what types of time-domain features are usually used in time series classification and list them. 

means, standard deviations, skewness, Kurtosis, max and min values

#### ii. Extract the time-domain features minimum, maximum, mean, median, standard deviation, first quartile, and third quartile for all of the 6 time series in each instance. 

In [316]:
# get all datasets
all_data = traindata + testdata

In [317]:
# define function to extract features
def add_feature(df, index, lst):
    minimum = df.min()[index]
    lst.append(minimum)
    maximum = df.max()[index]
    lst.append(maximum)
    mean = df.mean()[index]
    lst.append(mean)
    median = df.median()[index]
    lst.append(median)
    std = df.std()[index]
    lst.append(std)
    firstquart = df.quantile(q = 0.25)[index]
    lst.append(firstquart)
    thirdquart = df.quantile(q = 0.75)[index]
    lst.append(thirdquart) 

In [318]:
# get column names and time features names
cols=["min1","max1","mean1","median1","std1","1stQuart1","3rdQuart1",
     "min2","max2","mean2","median2","std2","1stQuart2","3rdQuart2", 
     "min3","max3","mean3","median3","std3","1stQuart3","3rdQuart3",
     "min4","max4","mean4","median4","std4","1stQuart4","3rdQuart4", 
     "min5","max5","mean5","median5","std5","1stQuart5","3rdQuart5", 
     "min6","max6","mean6","median6","std6","1stQuart6","3rdQuart6"]

time_series = ['avg_rss12','var_rss12','avg_rss13','var_rss13','avg_rss23','var_rss23']
feature_df = pd.DataFrame(columns = cols)

# extract features for each instance 
for df in all_data:
    lst = []
    for i in range(len(time_series)):
        add_feature(df, i, lst)
    s = pd.Series(lst, index=cols)
    feature_df = feature_df.append(s,ignore_index=True)

feature_df

,min1,max1,mean1,median1,std1,1stQuart1,3rdQuart1,min2,max2,mean2,...,std5,1stQuart5,3rdQuart5,min6,max6,mean6,median6,std6,1stQuart6,3rdQuart6
0,36.25,48.00,43.969125,44.500,1.618364,43.31,44.6700,0.0,1.50,0.413125,...,3.318301,20.5000,23.750,0.0,2.96,0.555313,0.490,0.487826,0.000,0.830
1,37.00,48.00,43.454958,43.250,1.386098,42.50,45.0000,0.0,1.58,0.378083,...,2.488862,22.2500,24.000,0.0,5.26,0.679646,0.500,0.622534,0.430,0.870
2,33.00,47.75,42.179813,43.500,3.670666,39.15,45.0000,0.0,3.00,0.696042,...,3.849448,30.4575,36.330,0.0,2.18,0.613521,0.500,0.524317,0.000,1.000
3,33.00,45.75,41.678063,41.750,2.243490,41.33,42.7500,0.0,2.83,0.535979,...,2.411026,28.4575,31.250,0.0,1.79,0.383292,0.430,0.389164,0.000,0.500
4,35.00,47.40,43.954500,44.330,1.558835,43.00,45.0000,0.0,1.70,0.426250,...,1.999604,35.3625,36.500,0.0,1.79,0.493292,0.430,0.513506,0.000,0.940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,35.50,46.25,43.174938,43.670,1.989052,42.50,44.5000,0.0,2.12,0.506583,...,2.983976,12.7500,16.500,0.0,5.72,0.911979,0.830,0.666161,0.470,1.220
84,32.75,47.00,42.760563,44.500,3.398919,41.33,45.3725,0.0,3.34,0.486167,...,4.296574,13.0000,18.565,0.0,5.73,0.842271,0.710,0.722165,0.430,1.090
85,19.33,43.50,34.227771,35.500,4.889576,30.50,37.7500,0.0,14.50,3.995729,...,3.092094,14.7500,18.670,0.0,9.74,3.394125,3.100,1.792090,2.105,4.425
86,12.50,45.00,33.509729,34.125,4.850923,30.50,36.7500,0.0,13.05,4.450771,...,3.133564,14.6275,18.750,0.0,8.96,3.378479,3.085,1.787360,2.060,4.440


#### iii. Estimate the standard deviation of each of the time-domain features you extracted from the data. Then, use Python’s bootstrapped or any other method to build a 90% bootsrap confidence interval for the standard deviation of each feature.

In [319]:
# get the standard deviation for each time_domain
feature_sd = feature_df.describe()[feature_df.describe().index == 'std'].T

In [320]:
# write function to change the feature names
index = list(feature_sd.index)
def get_feature_name(string):
    string_dict = {'1':'_avg_rss12', '2':'_var_rss12', '3':'_avg_rss13', 
                   '4':'_var_rss13', '5':'_avg_rss23', '6':'_var_rss23'}
    return string[:-1]+string_dict[list(string)[-1]]

In [321]:
# get new feature index
feature_lst = []
for i in index:
    feature_lst.append(get_feature_name(i))
    
feature_sd.index = feature_lst
feature_sd

,std
min_avg_rss12,9.569975
max_avg_rss12,4.394362
mean_avg_rss12,5.335718
median_avg_rss12,5.440054
std_avg_rss12,1.772153
1stQuart_avg_rss12,6.153590
3rdQuart_avg_rss12,5.138925
min_var_rss12,0.000000
max_var_rss12,5.062729
mean_var_rss12,1.574164


In [322]:
# build 90% bootstrap confidence interval with 1000 iterations
n_iterations = 1000
lower={}
upper={}
for i in feature_df.columns.tolist():
    bootstrap = []
    for p in range(n_iterations):
        sample = resample(feature_df[i])
        bootstrap.append(np.std(sample))
    bootstrap.sort()
    lower[i] = bootstrap[49]
    upper[i] = bootstrap[949]

In [323]:
# add lower bound and upper bound for each feature
lower_ci = pd.Series(lower)
lower_ci.index = feature_sd.index
upper_ci = pd.Series(upper)
upper_ci.index = feature_sd.index
feature_sd["lower bound"] = lower_ci
feature_sd["upper bound"] = upper_ci
feature_sd

,std,lower bound,upper bound
min_avg_rss12,9.569975,8.142286,10.672282
max_avg_rss12,4.394362,3.274749,5.269408
mean_avg_rss12,5.335718,4.656655,5.846540
median_avg_rss12,5.440054,4.748134,5.928860
std_avg_rss12,1.772153,1.567771,1.937674
1stQuart_avg_rss12,6.153590,5.533467,6.592781
3rdQuart_avg_rss12,5.138925,4.333611,5.800974
min_var_rss12,0.000000,0.000000,0.000000
max_var_rss12,5.062729,4.586937,5.372056
mean_var_rss12,1.574164,1.383009,1.688359


#### iv. Use your judgement to select the three most important time-domain features (one option may be min, mean, and max).

Mean: It gives the average or the most common value in the time series.
<br><br>Median: It measures the middle value of a sorted list of time series. One advantage is that it is not affected by extreme values.
<br><br>Standard Deviation: It shows how much data is clustered around a mean value. We can get an idea of how the time series data is distributed.

## 2. ISLR 3.7.4

#### (a) Suppose that the true relationship between X and Y is linear, i.e. Y = β0 + β1X + ϵ. Consider the training residual sum of squares (RSS) for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

We can expect that the training RSS for the cubic regression to be lower, since with the higher degree of freedom, the cubic regression model has higher level of flexibility. This means a decrease in the training mean squared error, which is the product of the number of observations and the training RSS. So, regardless of the true relationship, with the limited information, the training RSS for the cubic regression to be lower than that of the linear regression.

#### (b) Answer (a) using test rather than training RSS.

We can expect that the test RSS for linear regression model to be lower. According to the bias-variance tradeoff, as flexibility increases, we see an increase in variance and a decrease in bias. The linear regression model is less flexible than the cubic regression model, so the variance would be lower for linear regression. Also, the true relationship between X and Y is linear, which means the bias for the linear model would also be lower. A lower bias and a lower variance in linear regression model will result in a lower test RSS than that of the cubic regression.

#### (c) Suppose that the true relationship between X and Y is not linear, but we don’t know how far it is from linear. Consider the training RSS for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

There is not enough information to tell which model has the lower training RSS. Just like what I talk about in answer (a), it depends on the level of flexibility (degree of freedom). Since we do not know how far the true relationship is from linear, we cannot compare it with the training RSS of the cubic regression model.

#### (d) Answer (c) using test rather than training RSS.

There is not enough information to tell which model has the lower test RSS. Just like what I talk about in answer (c) and (b), it depends on the level of flexibility (degree of freedom) and the true relationship between X and Y. Since we do not know how far the true relationship is from linear, we cannot compare it with the test RSS of the cubic regression model.